In [94]:
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import plotly.offline as plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
yf.pdr_override()
import os
plotly.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'
import wrds
import numpy as np


## Specifying date range

In [95]:
# selecting dates to be 10 year window after Feng et al. 2012's paper
end = dt.datetime(2023, 1, 1)
start = dt.datetime(2013, 1, 1)
start, end

(datetime.datetime(2013, 1, 1, 0, 0), datetime.datetime(2023, 1, 1, 0, 0))

## Select stock/ticker

# Importing stock tickers from 
from https://stockmarketmba.com/stocksinthesp500.php


In [96]:
working_dir = os.getcwd()

In [97]:
os.listdir(working_dir)

['.ipynb_checkpoints',
 'data',
 'DataImport-1.1.ipynb',
 'DataImport-1.2.ipynb',
 'DataImport-1.3.ipynb',
 'DataImport-1.4.ipynb',
 'DataImport-1.5.ipynb',
 'DataImport-1.6.ipynb',
 'DataImport-1.7.ipynb',
 'DataImport-1.8.ipynb',
 'DataImport-1.9.ipynb',
 'DataImport-2.0.ipynb',
 'gridsearch.ipynb',
 'gridsearch_lsq.ipynb',
 'main - edits Nitai.ipynb',
 'main.ipynb',
 'model - edit Nitai.ipynb',
 'model+real_data_gridsearch_lsq.ipynb',
 'model.ipynb',
 'model_test.py',
 'ParameterTuning.ipynb',
 'PCS_Financial_Market_ABM-main.zip',
 'PCS_Update.ipynb',
 'README.md',
 'Real_stock_data_analysis.ipynb',
 'Real_stock_data_analysis_Calvin_Take.ipynb',
 'Reviewed6&7_by_Team5.docx',
 'Stock_Data_analysis.ipynb',
 'Stock_Data_analysis_gridsearch_test.ipynb',
 'Trail_Agent_code.ipynb',
 'used_ticker_list.txt']

In [98]:
path = "data/Stocks_in_SP_500_Index.xlsx"
data = pd.read_excel(path,skiprows=1)
data.head()

,Symbol,Description,Category2,Category3,GICS Sector,Market cap,Dividend yield,Price to TTM earnings,Price to TTM sales,Price to book value,Action
0,AAPL,Apple Inc,Common stocks,Large cap,Information Technology,1988832912360,0.0073,0.0,0.0,0.0,Analyze
1,MSFT,Microsoft Corp,Common stocks,Large cap,Information Technology,1657655067218,0.0114,0.0,0.0,0.0,Analyze
2,GOOG,Alphabet Inc Class C,Common stocks,Large cap,Communication Services,1042954820000,0.0000,0.0,0.0,0.0,Analyze
3,GOOGL,Alphabet Inc Class A,Common stocks,Large cap,Communication Services,1042954820000,0.0000,0.0,0.0,0.0,Analyze
4,AMZN,Amazon.Com Inc.,Common stocks,Large cap,Consumer Discretionary,847961495109,0.0000,0.0,0.0,0.0,Analyze


In [99]:
tickers = data['Symbol'].to_list()

In [100]:
tickers

['AAPL',
 'MSFT',
 'GOOG',
 'GOOGL',
 'AMZN',
 'BRK.B',
 'JNJ',
 'UNH',
 'XOM',
 'V',
 'JPM',
 'WMT',
 'PG',
 'NVDA',
 'TSLA',
 'META',
 'LLY',
 'CVX',
 'MA',
 'HD',
 'ABBV',
 'MRK',
 'PFE',
 'BAC',
 'KO',
 'PEP',
 'AVGO',
 'ORCL',
 'TMO',
 'COST',
 'ABT',
 'CSCO',
 'MCD',
 'NKE',
 'DHR',
 'TMUS',
 'VZ',
 'ACN',
 'DIS',
 'NEE',
 'WFC',
 'CMCSA',
 'PM',
 'BMY',
 'ADBE',
 'TXN',
 'SCHW',
 'LIN',
 'UPS',
 'RTX',
 'COP',
 'MS',
 'AMGN',
 'HON',
 'T',
 'NFLX',
 'CRM',
 'IBM',
 'CAT',
 'LMT',
 'UNP',
 'LOW',
 'DE',
 'QCOM',
 'BA',
 'SBUX',
 'CVS',
 'GS',
 'INTC',
 'ELV',
 'SPGI',
 'AXP',
 'GILD',
 'MDT',
 'INTU',
 'BLK',
 'PLD',
 'AMD',
 'AMT',
 'ADP',
 'ISRG',
 'SYK',
 'EL',
 'TJX',
 'CI',
 'CB',
 'C',
 'MDLZ',
 'PYPL',
 'BKNG',
 'AMAT',
 'MMC',
 'NOC',
 'ADI',
 'MO',
 'DUK',
 'GE',
 'REGN',
 'PGR',
 'SO',
 'SLB',
 'VRTX',
 'NOW',
 'EOG',
 'BDX',
 'TGT',
 'HCA',
 'ZTS',
 'MMM',
 'USB',
 'ITW',
 'GD',
 'APD',
 'MRNA',
 'CL',
 'BSX',
 'WM',
 'CSX',
 'PNC',
 'FISV',
 'ETN',
 'AON',
 'HUM',
 'E

## pandas_datareader module

In [123]:
# 1. pdr.DataReader(stocks, 'yahoo', start, end)
# 2. pdr.get_data_yahoo(stocks, start, end)

df = pdr.get_data_yahoo(tickers, start, end)

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BRK.B: No timezone found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


# Making a DataFrame containing Adjusted close Daily returns and volume returns from Yahoo finance

In [124]:
df.columns

MultiIndex([('Adj Close',    'A'),
            ('Adj Close',  'AAL'),
            ('Adj Close',  'AAP'),
            ('Adj Close', 'AAPL'),
            ('Adj Close', 'ABBV'),
            ('Adj Close',  'ABC'),
            ('Adj Close',  'ABT'),
            ('Adj Close', 'ACGL'),
            ('Adj Close',  'ACN'),
            ('Adj Close', 'ADBE'),
            ...
            (   'Volume', 'WYNN'),
            (   'Volume',  'XEL'),
            (   'Volume',  'XOM'),
            (   'Volume', 'XRAY'),
            (   'Volume',  'XYL'),
            (   'Volume',  'YUM'),
            (   'Volume',  'ZBH'),
            (   'Volume', 'ZBRA'),
            (   'Volume', 'ZION'),
            (   'Volume',  'ZTS')],
           length=3018)

In [125]:
#checking for missing values & printing the columns with missing values
col_with_missing_values = {}
for col in df['Close'].columns:
    # counting the columns with missing values in Adj Close
    sum_of_missing_values = df['Close'][col].isnull().sum()
    # also counting the columns with missing values in Volume
    sum_of_missing_values += df['Volume'][col].isnull().sum()
    if sum_of_missing_values > 0:
        col_with_missing_values[col] = sum_of_missing_values
        print(col)

ALLE
ANET
BF.B
BRK.B
CARR
CDAY
CDW
CEG
CFG
CTLT
CTVA
CZR
DOW
ETSY
FOX
FOXA
FTV
HLT
HPE
HWM
INVH
IQV
IR
KEYS
KHC
LW
MRNA
NCLH
NWS
NWSA
OGN
OTIS
PAYC
PYPL
QRVO
SEDG
SYF
VICI
WRK
ZTS


In [126]:
df_col_missing_values = pd.DataFrame(col_with_missing_values, index = [0]).T
df_col_missing_values.columns = ['Missing Values']

In [127]:
list_of_tickers_with_missing_values = df_col_missing_values.index.to_list()

In [128]:
print(f'stocks with missing values {list_of_tickers_with_missing_values}')
print(f'stocks to de removed: {len(list_of_tickers_with_missing_values)}')


stocks with missing values ['ALLE', 'ANET', 'BF.B', 'BRK.B', 'CARR', 'CDAY', 'CDW', 'CEG', 'CFG', 'CTLT', 'CTVA', 'CZR', 'DOW', 'ETSY', 'FOX', 'FOXA', 'FTV', 'HLT', 'HPE', 'HWM', 'INVH', 'IQV', 'IR', 'KEYS', 'KHC', 'LW', 'MRNA', 'NCLH', 'NWS', 'NWSA', 'OGN', 'OTIS', 'PAYC', 'PYPL', 'QRVO', 'SEDG', 'SYF', 'VICI', 'WRK', 'ZTS']
stocks to de removed: 40


In [129]:
df

Adj Close                                                 \
                     A        AAL         AAP        AAPL        ABBV   
Date                                                                    
2013-01-02   27.435247  13.179524   67.488426   16.862820   23.105906   
2013-01-03   27.533506  12.877847   67.488426   16.649979   22.915104   
2013-01-04   28.077234  13.886581   68.540474   16.186195   22.625629   
2013-01-07   27.874155  13.990283   68.307678   16.090981   22.671680   
2013-01-08   27.651424  14.291959   67.190475   16.134289   22.178246   
...                ...        ...         ...         ...         ...   
2022-12-23  149.007767  12.710000  143.279999  131.860001  161.564163   
2022-12-27  149.327301  12.530000  145.020004  130.029999  161.455200   
2022-12-28  147.869461  12.320000  145.300003  126.040001  160.702347   
2022-12-29  150.865005  12.700000  146.309998  129.610001  161.029236   
2022-12-30  149.649994  12.720000  147.029999  129.929993  160.088181   

                                                                       ...  \
                   ABC         ABT       ACGL         ACN        ADBE  ...   
Date                                                                   ...   
2013-01-02   37.189945   26.401579  14.793333   57.290283   38.340000  ...   
2013-01-03   37.112976   27.406569  14.750000   57.082886   37.750000  ...   
2013-01-04   37.360966   27.241823  14.876667   57.398132   38.130001  ...   
2013-01-07   37.480690   27.464235  14.730000   57.149258   37.939999  ...   
2013-01-08   37.429375   27.472471  14.750000   57.481079   38.139999  ...   
...                ...         ...        ...         ...         ...  ...   
2022-12-23  170.009995  107.692398  63.380001  265.006012  338.450012  ...   
2022-12-27  169.000000  108.080643  63.619999  264.229187  335.089996  ...   
2022-12-28  167.360001  107.343979  62.599998  262.048126  328.329987  ...   
2022-12-29  166.050003  109.812798  63.110001  267.286682  337.579987  ...   
2022-12-30  165.710007  109.295143  62.779999  265.752960  336.529999  ...   

             Volume                                                          \
               WYNN      XEL       XOM     XRAY      XYL       YUM      ZBH   
Date                                                                          
2013-01-02  2487000  2702000  16143700   856600   927300   6091328  1439425   
2013-01-03  1731900  2403600  13268200   525500   610800   3654574  1622250   
2013-01-04  2129100  1750100  11427900   660400   563400   3782685  1226524   
2013-01-07  1501000  2856800  11799800   554100   481300   5289417   985710   
2013-01-08   976300  3761500  14226400   698900  1084500  17253686   932356   
...             ...      ...       ...      ...      ...       ...      ...   
2022-12-23  1134800  1201300  11539400  1266800   321100    700800   789800   
2022-12-27  4026500  1580600  11962100   998800   468200   1300200   755600   
2022-12-28  2131500  1513700  10702100  1430500   480400    964800   750100   
2022-12-29  1431100  1398000  10534000  1139500   516300    875700   686600   
2022-12-30  1614300  1595700  11793300  1352700   548900   1401300   783800   

                                        
              ZBRA     ZION        ZTS  
Date                                    
2013-01-02  213800  2551100        NaN  
2013-01-03   99900  2267900        NaN  
2013-01-04  146000  3577700        NaN  
2013-01-07   90600  2286000        NaN  
2013-01-08  112600  3758600        NaN  
...            ...      ...        ...  
2022-12-23  180900  1263300  1017900.0  
2022-12-27  252500   704200   957900.0  
2022-12-28  241200   680300  1443900.0  
2022-12-29  274900   685300  1298900.0  
2022-12-30  228200   609700  1249500.0  

[2518 rows x 3018 columns]

In [130]:
df_close_copy = df['Close'].copy()
df_volume_copy = df['Volume'].copy()

In [136]:
df_close_copy.to_csv('df_close_copy')

In [131]:
# removing stocks with missing values
df_close_copy.drop(list_of_tickers_with_missing_values, axis=1, inplace=True)
df_volume_copy.drop(list_of_tickers_with_missing_values, axis=1, inplace=True)
df_close_clean = df_close_copy
df_volume_clean = df_volume_copy


In [132]:
df_close_clean

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,29.957083,13.98,72.489998,19.608213,35.119999,43.490002,32.049999,14.793333,69.059998,38.340000,...,29.150000,118.070000,27.379999,88.709999,40.520000,27.500000,48.281811,65.728157,40.959999,21.990000
2013-01-03,30.064379,13.66,72.489998,19.360714,34.830002,43.400002,33.270000,14.750000,68.809998,37.750000,...,29.150000,119.339996,27.299999,88.550003,40.410000,27.410000,48.583752,66.669907,41.000000,22.020000
2013-01-04,30.658083,14.73,73.620003,18.821428,34.389999,43.689999,33.070000,14.876667,69.190002,38.130001,...,29.389999,120.779999,27.400000,88.959999,40.759998,27.370001,49.115746,67.009712,40.669998,22.650000
2013-01-07,30.436338,14.84,73.370003,18.710714,34.459999,43.830002,33.340000,14.730000,68.889999,37.939999,...,29.480000,121.190002,27.110001,87.930000,40.900002,27.010000,48.806614,67.174759,40.900002,22.540001
2013-01-08,30.193132,15.16,72.169998,18.761070,33.709999,43.770000,33.349998,14.750000,69.290001,38.139999,...,29.680000,121.089996,27.160000,88.480003,40.430000,26.680000,46.757729,67.271843,40.930000,22.090000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,149.229996,12.71,143.279999,131.860001,163.100006,170.009995,108.180000,63.380001,266.089996,338.450012,...,31.490000,80.720001,70.930000,108.680000,31.830000,109.730003,128.899994,126.690002,248.220001,48.450001
2022-12-27,149.550003,12.53,145.020004,130.029999,162.990005,169.000000,108.570000,63.619999,265.309998,335.089996,...,31.469999,84.330002,71.570000,110.190002,32.070000,110.720001,129.899994,127.279999,251.000000,48.840000
2022-12-28,148.089996,12.32,145.300003,126.040001,162.229996,167.360001,107.830002,62.599998,263.119995,328.329987,...,30.629999,80.089996,70.570000,108.379997,30.980000,108.940002,129.309998,125.989998,246.839996,47.970001


In [133]:
df_volume_clean

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,8790205,6662000,800500,560518000,13767900,1972400,20266400,2932800,4040500,6483800,...,6595000,2487000,2702000,16143700,856600,927300,6091328,1439425,213800,2551100
2013-01-03,5751791,5398400,520600,352965200,16739300,2026700,22148200,1966200,3340700,3906000,...,3967200,1731900,2403600,13268200,525500,610800,3654574,1622250,99900,2267900
2013-01-04,6432897,12048300,615000,594333600,21372100,2461500,15820100,1591800,3145600,3809300,...,3639400,2129100,1750100,11427900,660400,563400,3782685,1226524,146000,3577700
2013-01-07,3589505,5730600,1054400,484156400,17897100,1803600,13120000,1296900,2262800,3632100,...,2460400,1501000,2856800,11799800,554100,481300,5289417,985710,90600,2286000
2013-01-08,3896925,8034400,840000,458707200,17863300,1533000,15042300,1810800,2502800,3080900,...,5202600,976300,3761500,14226400,698900,1084500,17253686,932356,112600,3758600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,779400,17181200,524500,63814900,2685100,528000,2618400,890300,1213300,1629800,...,1932800,1134800,1201300,11539400,1266800,321100,700800,789800,180900,1263300
2022-12-27,879500,18878200,717100,69007800,2669200,740300,2927400,666700,1009600,1464300,...,2266000,4026500,1580600,11962100,998800,468200,1300200,755600,252500,704200
2022-12-28,784300,20470400,685500,85438400,2944500,804700,3265200,1107900,1387000,1672100,...,2244100,2131500,1513700,10702100,1430500,480400,964800,750100,241200,680300


In [18]:
# taking Daily returns (percent change of  Adj close of the stocks)  and storing in a new dataframe
# also dropping the first row as it will be NaN after percent change transformaion


# usually returns would be described as pct change in price at time t vs. t-1, but this transformation will not be used since we found out feng et al. 2012 moddelled daily returns as log (xt+1) - log(xt) which we will do aswell.
# df_daily_returns_clean = df_adj_close_clean.apply(np.log).diff().dropna()

df_daily_returns_clean = df_adj_close_clean

In [19]:
used_ticker_symbols = df_daily_returns_clean.columns
used_ticker_symbols

Index(['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABT', 'ACGL', 'ACN', 'ADBE',
       ...
       'WY', 'WYNN', 'XEL', 'XOM', 'XRAY', 'XYL', 'YUM', 'ZBH', 'ZBRA',
       'ZION'],
      dtype='object', length=463)

In [21]:
with open(r'used_ticker_list.txt', 'w') as file:
    for item in used_ticker_symbols:
        file.write("%s\n" % item)
    print('done')

done


# loading WRDS Computat trading volume Data

In [22]:
# stock volume data row used belpw: https://www.dropbox.com/s/80r3e9ayeppz8ca/Stock_volume_WRDS_query_csv.zip?dl=0
# shares oustanding data row used belpw: https://www.dropbox.com/s/c9nf1lh3sdggsp0/Stock_sharesoutstanding_WRDS_query_csv.csv?dl=0

In [27]:
volume_stock_data = pd.read_csv(r'data\stock_volume_WRDS_query_csv.csv', parse_dates=True, index_col=2)

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.



In [28]:
volume_stock_data.columns

Index(['gvkey', 'iid', 'tic', 'conm', 'cshtrd'], dtype='object')

In [29]:
volume_stock_data

,gvkey,iid,tic,conm,cshtrd
datadate,,,,,
2007-01-03,1045,01,AAMRQ,AMERICAN AIRLINES GROUP INC,12096600.0
2007-01-04,1045,01,AAMRQ,AMERICAN AIRLINES GROUP INC,9454800.0
2007-01-05,1045,01,AAMRQ,AMERICAN AIRLINES GROUP INC,7861900.0
2007-01-08,1045,01,AAMRQ,AMERICAN AIRLINES GROUP INC,7896300.0
2007-01-09,1045,01,AAMRQ,AMERICAN AIRLINES GROUP INC,7045300.0
...,...,...,...,...,...
2010-11-30,294524,2,LYB.B,LYONDELLBASELL INDUSTRIES NV,310980.0
2010-12-01,294524,2,LYB.B,LYONDELLBASELL INDUSTRIES NV,832088.0
2010-12-02,294524,2,LYB.B,LYONDELLBASELL INDUSTRIES NV,451115.0


In [30]:
volume_stock_data

,gvkey,iid,tic,conm,cshtrd
datadate,,,,,
2007-01-03,1045,01,AAMRQ,AMERICAN AIRLINES GROUP INC,12096600.0
2007-01-04,1045,01,AAMRQ,AMERICAN AIRLINES GROUP INC,9454800.0
2007-01-05,1045,01,AAMRQ,AMERICAN AIRLINES GROUP INC,7861900.0
2007-01-08,1045,01,AAMRQ,AMERICAN AIRLINES GROUP INC,7896300.0
2007-01-09,1045,01,AAMRQ,AMERICAN AIRLINES GROUP INC,7045300.0
...,...,...,...,...,...
2010-11-30,294524,2,LYB.B,LYONDELLBASELL INDUSTRIES NV,310980.0
2010-12-01,294524,2,LYB.B,LYONDELLBASELL INDUSTRIES NV,832088.0
2010-12-02,294524,2,LYB.B,LYONDELLBASELL INDUSTRIES NV,451115.0


In [31]:
volume_stock_data = volume_stock_data.drop(columns=['gvkey', 'iid', 'conm'])

In [32]:
volume_stock_data

,tic,cshtrd
datadate,,
2007-01-03,AAMRQ,12096600.0
2007-01-04,AAMRQ,9454800.0
2007-01-05,AAMRQ,7861900.0
2007-01-08,AAMRQ,7896300.0
2007-01-09,AAMRQ,7045300.0
...,...,...
2010-11-30,LYB.B,310980.0
2010-12-01,LYB.B,832088.0
2010-12-02,LYB.B,451115.0


In [33]:
volume_stock_data['tic'].nunique()

970

In [34]:
volume_stock_data.tic.nunique()

970

In [35]:
volume_stock_data['Date'] = volume_stock_data.index

In [36]:
volume_stock_data

,tic,cshtrd,Date
datadate,,,
2007-01-03,AAMRQ,12096600.0,2007-01-03
2007-01-04,AAMRQ,9454800.0,2007-01-04
2007-01-05,AAMRQ,7861900.0,2007-01-05
2007-01-08,AAMRQ,7896300.0,2007-01-08
2007-01-09,AAMRQ,7045300.0,2007-01-09
...,...,...,...
2010-11-30,LYB.B,310980.0,2010-11-30
2010-12-01,LYB.B,832088.0,2010-12-01
2010-12-02,LYB.B,451115.0,2010-12-02


In [37]:
volume_stock_data_clean_pivot = volume_stock_data.pivot(index = 'Date', columns='tic',values = 'cshtrd')

In [38]:
volume_stock_data_clean_pivot

tic,0050B,0051B,1974B,2091B,3CCIKO,4741B,4764B,5050B,5235B,5236B,...,ZB.PF,ZBH,ZBH.WI,ZBRA,ZION,ZIONN,ZIONO,ZIONP,ZIONW,ZIONZ
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-03,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,1792300.0,NaN,560760.0,693041.0,NaN,NaN,0.0,NaN,NaN
2007-01-04,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,2053000.0,NaN,355633.0,569895.0,NaN,NaN,0.0,NaN,NaN
2007-01-05,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,1220400.0,NaN,645339.0,829044.0,NaN,NaN,500.0,NaN,NaN
2007-01-08,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,1806100.0,NaN,318215.0,620257.0,NaN,NaN,12700.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1423625.0,NaN,535362.0,902684.0,NaN,7514.0,2320.0,NaN,NaN
2023-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1894247.0,NaN,353624.0,878534.0,NaN,6911.0,1305.0,NaN,NaN
2023-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2448268.0,NaN,600101.0,697494.0,NaN,3202.0,1295.0,NaN,NaN


In [39]:
volume_stock_data_clean_pivot.shape

(4071, 970)

In [40]:
for col in volume_stock_data_clean_pivot.columns:
    print(col, volume_stock_data_clean_pivot[col].isnull().sum())

0050B 4071
0051B 4071
1974B 4071
2091B 4071
3CCIKO 2741
4741B 4055
4764B 4071
5050B 3490
5235B 4071
5236B 4071
5237B 4071
5238B 4071
6525B 4068
9297B 4071
A 34
AAL 1780
AALCP 3988
AAMRQ 2325
AAP 34
AAPL 34
AAR 2809
ABBV 1529
ABC 34
ABT 34
ACE.PC 3707
ACGL 34
ACGLN 3663
ACGLO 2707
ACGLP 2814
ACN 34
ADBE 34
ADI 34
ADM 34
ADM.PA 3314
ADP 34
ADSK 34
AEE 34
AEP 34
AEP.PA 2871
AEPPL 3322
AEPPZ 3465
AES 34
AESC 3607
AFL 34
AHC.2 4071
AIG 34
AIG.PA 3104
AIG.PA.1 3264
AIG.WS 1554
AIZ 34
AIZP 3312
AJG 34
AKAM 34
ALB 34
ALGN 34
ALK 34
ALL 34
ALL.PA 2409
ALL.PC 2799
ALL.PD 2610
ALL.PE 2652
ALL.PF 2725
ALL.PG 2863
ALL.PH 3206
ALL.PI 3273
AMAT 34
AMCO30 962
AMCR 3164
AMD 34
AME 34
AMGN 34
AMP 34
AMP.PA 2806
AMT 34
AMT.PA 3311
AMT.PB 3322
AMZN 34
ANSS 34
ANTX.2 3324
AON 34
AOS 34
APA 34
APA.PD 3310
APD 34
APH 34
APTV 1264
APTV.PA 3420
ARE 34
ARE.PB 4019
ARE.PC 2742
ARE.PD 1195
ARE.PE 2788
ARH.PA 2728
ARH.PB 2728
ARH.PC 2628
ATO 34
ATVI 416
AVB 34
AVB.PH 3621
AVGO 687
AVGOP 3311
AVY 34
AWK 362
AXP 34


In [41]:
list_of_missing = []
for col in volume_stock_data_clean_pivot.columns:
    list_of_missing.append((str(col), volume_stock_data_clean_pivot[col].isnull().sum()))

In [42]:
print(list_of_missing)

[('0050B', 4071), ('0051B', 4071), ('1974B', 4071), ('2091B', 4071), ('3CCIKO', 2741), ('4741B', 4055), ('4764B', 4071), ('5050B', 3490), ('5235B', 4071), ('5236B', 4071), ('5237B', 4071), ('5238B', 4071), ('6525B', 4068), ('9297B', 4071), ('A', 34), ('AAL', 1780), ('AALCP', 3988), ('AAMRQ', 2325), ('AAP', 34), ('AAPL', 34), ('AAR', 2809), ('ABBV', 1529), ('ABC', 34), ('ABT', 34), ('ACE.PC', 3707), ('ACGL', 34), ('ACGLN', 3663), ('ACGLO', 2707), ('ACGLP', 2814), ('ACN', 34), ('ADBE', 34), ('ADI', 34), ('ADM', 34), ('ADM.PA', 3314), ('ADP', 34), ('ADSK', 34), ('AEE', 34), ('AEP', 34), ('AEP.PA', 2871), ('AEPPL', 3322), ('AEPPZ', 3465), ('AES', 34), ('AESC', 3607), ('AFL', 34), ('AHC.2', 4071), ('AIG', 34), ('AIG.PA', 3104), ('AIG.PA.1', 3264), ('AIG.WS', 1554), ('AIZ', 34), ('AIZP', 3312), ('AJG', 34), ('AKAM', 34), ('ALB', 34), ('ALGN', 34), ('ALK', 34), ('ALL', 34), ('ALL.PA', 2409), ('ALL.PC', 2799), ('ALL.PD', 2610), ('ALL.PE', 2652), ('ALL.PF', 2725), ('ALL.PG', 2863), ('ALL.PH', 3

In [43]:
sorted(list_of_missing, key= lambda x: x[1], reverse=True)

[('0050B', 4071),
 ('0051B', 4071),
 ('1974B', 4071),
 ('2091B', 4071),
 ('4764B', 4071),
 ('5235B', 4071),
 ('5236B', 4071),
 ('5237B', 4071),
 ('5238B', 4071),
 ('9297B', 4071),
 ('AHC.2', 4071),
 ('GM.1', 4071),
 ('GMM.U', 4071),
 ('PLP.Z', 4071),
 ('PXD.', 4071),
 ('6525B', 4068),
 ('SWP', 4068),
 ('MRK.WI', 4066),
 ('ZBH.WI', 4066),
 ('DOV.WI', 4063),
 ('VNO.WI', 4063),
 ('HON.WI', 4061),
 ('PSA.PT.1', 4061),
 ('4741B', 4055),
 ('PSA.PU.1', 4039),
 ('LNC.PD', 4031),
 ('ARE.PB', 4019),
 ('BAC.PS.1', 4017),
 ('DALRQ', 3991),
 ('AALCP', 3988),
 ('NEE.PR', 3988),
 ('EPB.2', 3977),
 ('UDR.PB', 3971),
 ('KMI.1', 3969),
 ('KEY.PL', 3968),
 ('C.PF.', 3957),
 ('NEMUF', 3956),
 ('MS.PP', 3954),
 ('SBT.1', 3944),
 ('DG.1', 3943),
 ('C.PG.', 3942),
 ('EQR.PD', 3938),
 ('KEY.PH', 3934),
 ('LYB.B', 3921),
 ('NIMC', 3917),
 ('RJF.PA', 3915),
 ('RJF.PB', 3915),
 ('EMN.WS', 3912),
 ('PSA.PR', 3900),
 ('PSA.PS', 3900),
 ('C.PH.1', 3899),
 ('MS.PO', 3899),
 ('FRC.1', 3889),
 ('FRC.PA.1', 3888),
 ('F

In [44]:
list_of_missing

[('0050B', 4071),
 ('0051B', 4071),
 ('1974B', 4071),
 ('2091B', 4071),
 ('3CCIKO', 2741),
 ('4741B', 4055),
 ('4764B', 4071),
 ('5050B', 3490),
 ('5235B', 4071),
 ('5236B', 4071),
 ('5237B', 4071),
 ('5238B', 4071),
 ('6525B', 4068),
 ('9297B', 4071),
 ('A', 34),
 ('AAL', 1780),
 ('AALCP', 3988),
 ('AAMRQ', 2325),
 ('AAP', 34),
 ('AAPL', 34),
 ('AAR', 2809),
 ('ABBV', 1529),
 ('ABC', 34),
 ('ABT', 34),
 ('ACE.PC', 3707),
 ('ACGL', 34),
 ('ACGLN', 3663),
 ('ACGLO', 2707),
 ('ACGLP', 2814),
 ('ACN', 34),
 ('ADBE', 34),
 ('ADI', 34),
 ('ADM', 34),
 ('ADM.PA', 3314),
 ('ADP', 34),
 ('ADSK', 34),
 ('AEE', 34),
 ('AEP', 34),
 ('AEP.PA', 2871),
 ('AEPPL', 3322),
 ('AEPPZ', 3465),
 ('AES', 34),
 ('AESC', 3607),
 ('AFL', 34),
 ('AHC.2', 4071),
 ('AIG', 34),
 ('AIG.PA', 3104),
 ('AIG.PA.1', 3264),
 ('AIG.WS', 1554),
 ('AIZ', 34),
 ('AIZP', 3312),
 ('AJG', 34),
 ('AKAM', 34),
 ('ALB', 34),
 ('ALGN', 34),
 ('ALK', 34),
 ('ALL', 34),
 ('ALL.PA', 2409),
 ('ALL.PC', 2799),
 ('ALL.PD', 2610),
 ('ALL.

In [45]:
volume_stock_data_clean_pivot

tic,0050B,0051B,1974B,2091B,3CCIKO,4741B,4764B,5050B,5235B,5236B,...,ZB.PF,ZBH,ZBH.WI,ZBRA,ZION,ZIONN,ZIONO,ZIONP,ZIONW,ZIONZ
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-03,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,1792300.0,NaN,560760.0,693041.0,NaN,NaN,0.0,NaN,NaN
2007-01-04,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,2053000.0,NaN,355633.0,569895.0,NaN,NaN,0.0,NaN,NaN
2007-01-05,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,1220400.0,NaN,645339.0,829044.0,NaN,NaN,500.0,NaN,NaN
2007-01-08,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,1806100.0,NaN,318215.0,620257.0,NaN,NaN,12700.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1423625.0,NaN,535362.0,902684.0,NaN,7514.0,2320.0,NaN,NaN
2023-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1894247.0,NaN,353624.0,878534.0,NaN,6911.0,1305.0,NaN,NaN
2023-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2448268.0,NaN,600101.0,697494.0,NaN,3202.0,1295.0,NaN,NaN


In [46]:
volume_stock_data_clean_pivot= volume_stock_data_clean_pivot[start:end][used_ticker_symbols]

In [47]:
# we will fill the missing values with the previous day's value, otherwise we need to throw away to much data

volume_stock_data_clean_pivot.fillna(method='backfill', inplace=True)

In [48]:
volume_stock_data_clean_pivot 

tic,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,6290038.0,43167060.0,800421.0,19986670.0,13767660.0,1972401.0,20266410.0,977536.0,4039095.0,6483720.0,...,6594952.0,2486815.0,2702053.0,16144970.0,856539.0,927252.0,4376855.0,1397462.0,213748.0,2551059.0
2013-01-03,4114289.0,43167060.0,520539.0,12579170.0,16739210.0,2027242.0,22148090.0,655362.0,3340674.0,3905904.0,...,3967047.0,1731879.0,2403510.0,13268470.0,525445.0,610706.0,2627253.0,1574966.0,99849.0,2267804.0
2013-01-04,4601506.0,43167060.0,614933.0,21196320.0,21372200.0,2462183.0,15819140.0,530563.0,3145680.0,3809146.0,...,3624749.0,2129025.0,1743995.0,11427870.0,660389.0,563325.0,2719375.0,1189537.0,145944.0,3577660.0
2013-01-07,2567557.0,43167060.0,1051969.0,17262620.0,17896920.0,1803504.0,13121470.0,423123.0,2262426.0,3632022.0,...,2450680.0,1500928.0,2853210.0,11799780.0,554059.0,481287.0,3729703.0,954238.0,90581.0,2285917.0
2013-01-08,2787813.0,43167060.0,840003.0,16350190.0,17863000.0,1533163.0,15053860.0,580903.0,2468876.0,3080868.0,...,5203250.0,975921.0,3762904.0,14228400.0,698803.0,1084482.0,12406140.0,905965.0,112590.0,3758558.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,779426.0,17160810.0,524534.0,63755280.0,2684724.0,528005.0,2618369.0,890265.0,1213251.0,1627915.0,...,1932785.0,1134475.0,1201346.0,11534290.0,1264320.0,321094.0,699735.0,789770.0,180932.0,1263345.0
2022-12-27,879543.0,18836880.0,717073.0,68887690.0,2669090.0,740310.0,2927352.0,666705.0,1009563.0,1463215.0,...,2265992.0,4024281.0,1580610.0,11874640.0,998778.0,468173.0,1299183.0,742499.0,252510.0,704184.0
2022-12-28,784312.0,20388570.0,685513.0,85247210.0,2944327.0,804725.0,3264409.0,1107929.0,1386969.0,1668511.0,...,2244112.0,2131050.0,1513689.0,10675270.0,1430535.0,480378.0,964755.0,750108.0,241209.0,679957.0


In [49]:
df_volume_clean_wrds = volume_stock_data_clean_pivot

In [50]:
# removing rows from df_volume_clean_wrds to match df_volume_clean, so  that dataframes are same shape. using both yahoo finance and WRDS Computat volumes data to see if there is a difference in the results
mask_3 = volume_stock_data_clean_pivot.index.isin(df_daily_returns_clean.index)
df_volume_clean_wrds = df_volume_clean_wrds[mask_3]


In [51]:
# reference: Wharton Research Data Services. "WRDS" wrds.wharton.upenn.edu, accessed 2023-01-16.

# loading WRDS Computat shares outanding Data

In [52]:
# stock volume data row used belpw: https://www.dropbox.com/s/80r3e9ayeppz8ca/Stock_volume_WRDS_query_csv.zip?dl=0
# shares oustanding data row used belpw: https://www.dropbox.com/s/c9nf1lh3sdggsp0/Stock_sharesoutstanding_WRDS_query_csv.csv?dl=0

In [55]:
shares_outstanding_stock_data = pd.read_csv(r'data\Stock_sharesoutstanding_WRDS_query_csv.csv', parse_dates=True, index_col=2)

In [56]:
shares_outstanding_stock_data.columns

Index(['gvkey', 'iid', 'tic', 'cshoc'], dtype='object')

In [57]:
shares_outstanding_stock_data

,gvkey,iid,tic,cshoc
datadate,,,,
2008-01-02,1045,01,AAMRQ,249122000.0
2008-01-03,1045,01,AAMRQ,249122000.0
2008-01-04,1045,01,AAMRQ,249122000.0
2008-01-07,1045,01,AAMRQ,249122000.0
2008-01-08,1045,01,AAMRQ,249122000.0
...,...,...,...,...
2010-11-30,294524,2,LYB.B,263902000.0
2010-12-01,294524,2,LYB.B,263902000.0
2010-12-02,294524,2,LYB.B,263902000.0


In [58]:
shares_outstanding_stock_data

,gvkey,iid,tic,cshoc
datadate,,,,
2008-01-02,1045,01,AAMRQ,249122000.0
2008-01-03,1045,01,AAMRQ,249122000.0
2008-01-04,1045,01,AAMRQ,249122000.0
2008-01-07,1045,01,AAMRQ,249122000.0
2008-01-08,1045,01,AAMRQ,249122000.0
...,...,...,...,...
2010-11-30,294524,2,LYB.B,263902000.0
2010-12-01,294524,2,LYB.B,263902000.0
2010-12-02,294524,2,LYB.B,263902000.0


In [59]:
shares_outstanding_stock_data = shares_outstanding_stock_data.drop(columns=['gvkey', 'iid'])

In [60]:
shares_outstanding_stock_data

,tic,cshoc
datadate,,
2008-01-02,AAMRQ,249122000.0
2008-01-03,AAMRQ,249122000.0
2008-01-04,AAMRQ,249122000.0
2008-01-07,AAMRQ,249122000.0
2008-01-08,AAMRQ,249122000.0
...,...,...
2010-11-30,LYB.B,263902000.0
2010-12-01,LYB.B,263902000.0
2010-12-02,LYB.B,263902000.0


In [61]:
shares_outstanding_stock_data['tic'].nunique()

949

In [62]:
shares_outstanding_stock_data.tic.nunique()

949

In [63]:
shares_outstanding_stock_data[ 'Date'] = shares_outstanding_stock_data.index

In [64]:
shares_outstanding_stock_data

,tic,cshoc,Date
datadate,,,
2008-01-02,AAMRQ,249122000.0,2008-01-02
2008-01-03,AAMRQ,249122000.0,2008-01-03
2008-01-04,AAMRQ,249122000.0,2008-01-04
2008-01-07,AAMRQ,249122000.0,2008-01-07
2008-01-08,AAMRQ,249122000.0,2008-01-08
...,...,...,...
2010-11-30,LYB.B,263902000.0,2010-11-30
2010-12-01,LYB.B,263902000.0,2010-12-01
2010-12-02,LYB.B,263902000.0,2010-12-02


In [65]:
shares_outstanding_stock_data_clean_pivot = shares_outstanding_stock_data.pivot(index = 'Date', columns='tic',values = 'cshoc')

In [66]:
shares_outstanding_stock_data_clean_pivot

tic,0050B,0051B,1974B,2091B,3CCIKO,4741B,4764B,5050B,5235B,5236B,...,ZB.PF,ZBH,ZBH.WI,ZBRA,ZION,ZIONN,ZIONO,ZIONP,ZIONW,ZIONZ
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,NaN,NaN,NaN,NaN,6361000.0,NaN,NaN,8255000.0,NaN,NaN,...,NaN,234725000.0,NaN,68017000.0,106980000.0,NaN,NaN,9600000.0,NaN,NaN
2008-01-03,NaN,NaN,NaN,NaN,6361000.0,NaN,NaN,8255000.0,NaN,NaN,...,NaN,234725000.0,NaN,68017000.0,106980000.0,NaN,NaN,9600000.0,NaN,NaN
2008-01-04,NaN,NaN,NaN,NaN,6361000.0,NaN,NaN,8255000.0,NaN,NaN,...,NaN,234725000.0,NaN,68017000.0,106980000.0,NaN,NaN,9600000.0,NaN,NaN
2008-01-07,NaN,NaN,NaN,NaN,6361000.0,NaN,NaN,8255000.0,NaN,NaN,...,NaN,234725000.0,NaN,68017000.0,106980000.0,NaN,NaN,9600000.0,NaN,NaN
2008-01-08,NaN,NaN,NaN,NaN,6361000.0,NaN,NaN,8255000.0,NaN,NaN,...,NaN,234725000.0,NaN,68017000.0,106980000.0,NaN,NaN,9600000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,209852000.0,NaN,51630000.0,149618000.0,NaN,5536000.0,2646000.0,NaN,NaN
2023-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,209852000.0,NaN,51630000.0,149618000.0,NaN,5536000.0,2646000.0,NaN,NaN
2023-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,209852000.0,NaN,51630000.0,149618000.0,NaN,5536000.0,2646000.0,NaN,NaN


In [67]:
shares_outstanding_stock_data_clean_pivot.shape

(3815, 949)

In [68]:
for col in volume_stock_data_clean_pivot.columns:
    print(col, volume_stock_data_clean_pivot[col].isnull().sum())

A 0
AAL 0
AAP 0
AAPL 0
ABBV 0
ABC 0
ABT 0
ACGL 0
ACN 0
ADBE 0
ADI 0
ADM 0
ADP 0
ADSK 0
AEE 0
AEP 0
AES 0
AFL 0
AIG 0
AIZ 0
AJG 0
AKAM 0
ALB 0
ALGN 0
ALK 0
ALL 0
AMAT 0
AMCR 0
AMD 0
AME 0
AMGN 0
AMP 0
AMT 0
AMZN 0
ANSS 0
AON 0
AOS 0
APA 0
APD 0
APH 0
APTV 0
ARE 0
ATO 0
ATVI 0
AVB 0
AVGO 0
AVY 0
AWK 0
AXP 0
AZO 0
BA 0
BAC 0
BALL 0
BAX 0
BBWI 0
BBY 0
BDX 0
BEN 0
BIIB 0
BIO 0
BK 0
BKNG 0
BKR 0
BLK 0
BMY 0
BR 0
BRO 0
BSX 0
BWA 0
BXP 0
C 0
CAG 0
CAH 0
CAT 0
CB 0
CBOE 0
CBRE 0
CCI 0
CCL 0
CDNS 0
CE 0
CF 0
CHD 0
CHRW 0
CHTR 0
CI 0
CINF 0
CL 0
CLX 0
CMA 0
CMCSA 0
CME 0
CMG 0
CMI 0
CMS 0
CNC 0
CNP 0
COF 0
COO 0
COP 0
COST 0
CPB 0
CPRT 0
CPT 0
CRL 0
CRM 0
CSCO 0
CSGP 0
CSX 0
CTAS 0
CTRA 0
CTSH 0
CVS 0
CVX 0
D 0
DAL 0
DD 0
DE 0
DFS 0
DG 0
DGX 0
DHI 0
DHR 0
DIS 0
DISH 0
DLR 0
DLTR 0
DOV 0
DPZ 0
DRI 0
DTE 0
DUK 0
DVA 0
DVN 0
DXC 0
DXCM 0
EA 0
EBAY 0
ECL 0
ED 0
EFX 0
EIX 0
EL 0
ELV 0
EMN 0
EMR 0
ENPH 0
EOG 0
EPAM 0
EQIX 0
EQR 0
EQT 0
ES 0
ESS 0
ETN 0
ETR 0
EVRG 0
EW 0
EXC 0
EXPD 0
EXPE 0
EXR 0
F 0
FA

In [69]:
list_of_missing = []
for col in shares_outstanding_stock_data_clean_pivot.columns:
    list_of_missing.append((str(col), shares_outstanding_stock_data_clean_pivot[col].isnull().sum()))

In [70]:
print(list_of_missing)

[('0050B', 3815), ('0051B', 3815), ('1974B', 3815), ('2091B', 3815), ('3CCIKO', 2736), ('4741B', 3799), ('4764B', 3815), ('5050B', 3485), ('5235B', 3815), ('5236B', 3815), ('5237B', 3815), ('5238B', 3815), ('6525B', 3812), ('9297B', 3815), ('A', 29), ('AAL', 1524), ('AALCP', 3734), ('AAMRQ', 2320), ('AAP', 29), ('AAPL', 29), ('AAR', 2804), ('ABBV', 1273), ('ABC', 29), ('ABT', 29), ('ACE.PC', 3702), ('ACGL', 29), ('ACGLN', 3407), ('ACGLO', 2452), ('ACGLP', 2573), ('ACN', 29), ('ADBE', 29), ('ADI', 29), ('ADM', 29), ('ADM.PA', 3058), ('ADP', 29), ('ADSK', 29), ('AEE', 29), ('AEP', 29), ('AEP.PA', 2616), ('AEPPL', 3066), ('AEPPZ', 3209), ('AES', 29), ('AESC', 3351), ('AFL', 29), ('AHC.2', 3815), ('AIG', 29), ('AIG.PA', 2848), ('AIG.PA.1', 3008), ('AIG.WS', 3815), ('AIZ', 29), ('AIZP', 3057), ('AJG', 29), ('AKAM', 29), ('ALB', 29), ('ALGN', 29), ('ALK', 29), ('ALL', 29), ('ALL.PA', 2153), ('ALL.PC', 2543), ('ALL.PD', 2354), ('ALL.PE', 2396), ('ALL.PF', 2469), ('ALL.PG', 2608), ('ALL.PH', 2

In [71]:
sorted(list_of_missing, key= lambda x: x[1], reverse=True)

[('0050B', 3815),
 ('0051B', 3815),
 ('1974B', 3815),
 ('2091B', 3815),
 ('4764B', 3815),
 ('5235B', 3815),
 ('5236B', 3815),
 ('5237B', 3815),
 ('5238B', 3815),
 ('9297B', 3815),
 ('AHC.2', 3815),
 ('AIG.WS', 3815),
 ('BAC.WA', 3815),
 ('BAC.WB', 3815),
 ('C.WA', 3815),
 ('CELG.R', 3815),
 ('CGBBW', 3815),
 ('CMA.WS', 3815),
 ('COF.WS', 3815),
 ('DOV.WI', 3815),
 ('EMN.WS', 3815),
 ('EXPEW', 3815),
 ('EXPEZ', 3815),
 ('F.WS', 3815),
 ('GM.1', 3815),
 ('GM.WA', 3815),
 ('GM.WB', 3815),
 ('GM.WC', 3815),
 ('GMM.U', 3815),
 ('HIG.WT', 3815),
 ('HON.WI', 3815),
 ('IACIW', 3815),
 ('JPM.WS', 3815),
 ('KMI.WS', 3815),
 ('LNC.WS', 3815),
 ('MRK.WI', 3815),
 ('MTB.WS', 3815),
 ('NEMUF', 3815),
 ('OXY.WS', 3815),
 ('PLP.Z', 3815),
 ('PNC.WS', 3815),
 ('PXD.', 3815),
 ('SBNYW', 3815),
 ('VNO.WI', 3815),
 ('WFC.WS', 3815),
 ('ZIONW', 3815),
 ('ZIONZ', 3815),
 ('ZBH.WI', 3814),
 ('6525B', 3812),
 ('SWP', 3811),
 ('4741B', 3799),
 ('LNC.PD', 3775),
 ('BAC.PS.1', 3762),
 ('AALCP', 3734),
 ('NEE.PR'

In [72]:
list_of_missing

[('0050B', 3815),
 ('0051B', 3815),
 ('1974B', 3815),
 ('2091B', 3815),
 ('3CCIKO', 2736),
 ('4741B', 3799),
 ('4764B', 3815),
 ('5050B', 3485),
 ('5235B', 3815),
 ('5236B', 3815),
 ('5237B', 3815),
 ('5238B', 3815),
 ('6525B', 3812),
 ('9297B', 3815),
 ('A', 29),
 ('AAL', 1524),
 ('AALCP', 3734),
 ('AAMRQ', 2320),
 ('AAP', 29),
 ('AAPL', 29),
 ('AAR', 2804),
 ('ABBV', 1273),
 ('ABC', 29),
 ('ABT', 29),
 ('ACE.PC', 3702),
 ('ACGL', 29),
 ('ACGLN', 3407),
 ('ACGLO', 2452),
 ('ACGLP', 2573),
 ('ACN', 29),
 ('ADBE', 29),
 ('ADI', 29),
 ('ADM', 29),
 ('ADM.PA', 3058),
 ('ADP', 29),
 ('ADSK', 29),
 ('AEE', 29),
 ('AEP', 29),
 ('AEP.PA', 2616),
 ('AEPPL', 3066),
 ('AEPPZ', 3209),
 ('AES', 29),
 ('AESC', 3351),
 ('AFL', 29),
 ('AHC.2', 3815),
 ('AIG', 29),
 ('AIG.PA', 2848),
 ('AIG.PA.1', 3008),
 ('AIG.WS', 3815),
 ('AIZ', 29),
 ('AIZP', 3057),
 ('AJG', 29),
 ('AKAM', 29),
 ('ALB', 29),
 ('ALGN', 29),
 ('ALK', 29),
 ('ALL', 29),
 ('ALL.PA', 2153),
 ('ALL.PC', 2543),
 ('ALL.PD', 2354),
 ('ALL.

In [73]:
shares_outstanding_stock_data_clean_pivot

tic,0050B,0051B,1974B,2091B,3CCIKO,4741B,4764B,5050B,5235B,5236B,...,ZB.PF,ZBH,ZBH.WI,ZBRA,ZION,ZIONN,ZIONO,ZIONP,ZIONW,ZIONZ
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,NaN,NaN,NaN,NaN,6361000.0,NaN,NaN,8255000.0,NaN,NaN,...,NaN,234725000.0,NaN,68017000.0,106980000.0,NaN,NaN,9600000.0,NaN,NaN
2008-01-03,NaN,NaN,NaN,NaN,6361000.0,NaN,NaN,8255000.0,NaN,NaN,...,NaN,234725000.0,NaN,68017000.0,106980000.0,NaN,NaN,9600000.0,NaN,NaN
2008-01-04,NaN,NaN,NaN,NaN,6361000.0,NaN,NaN,8255000.0,NaN,NaN,...,NaN,234725000.0,NaN,68017000.0,106980000.0,NaN,NaN,9600000.0,NaN,NaN
2008-01-07,NaN,NaN,NaN,NaN,6361000.0,NaN,NaN,8255000.0,NaN,NaN,...,NaN,234725000.0,NaN,68017000.0,106980000.0,NaN,NaN,9600000.0,NaN,NaN
2008-01-08,NaN,NaN,NaN,NaN,6361000.0,NaN,NaN,8255000.0,NaN,NaN,...,NaN,234725000.0,NaN,68017000.0,106980000.0,NaN,NaN,9600000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,209852000.0,NaN,51630000.0,149618000.0,NaN,5536000.0,2646000.0,NaN,NaN
2023-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,209852000.0,NaN,51630000.0,149618000.0,NaN,5536000.0,2646000.0,NaN,NaN
2023-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,209852000.0,NaN,51630000.0,149618000.0,NaN,5536000.0,2646000.0,NaN,NaN


In [74]:
shares_outstanding_stock_data_clean_pivot= shares_outstanding_stock_data_clean_pivot[start:end][used_ticker_symbols]

In [75]:
# we will fill the missing values with the previous day's value, otherwise we need to throw away to much data

shares_outstanding_stock_data_clean_pivot.fillna(method='backfill', inplace=True)

In [76]:
df_shares_outstanding_clean = shares_outstanding_stock_data_clean_pivot
df_shares_outstanding_clean

tic,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,347935000.0,219297000.0,73364000.0,9.388190e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100824000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
2013-01-03,347935000.0,219297000.0,73364000.0,9.388190e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100824000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
2013-01-04,347935000.0,219297000.0,73364000.0,9.388190e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100867000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
2013-01-07,347935000.0,219297000.0,73364000.0,9.388190e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100867000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
2013-01-08,347935000.0,219297000.0,73364000.0,9.390390e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100867000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,296072000.0,649901000.0,59254000.0,1.590812e+10,1.768481e+09,205669000.0,1.743574e+09,369873000.0,658389000.0,464900000.0,...,735917000.0,113314000.0,547248000.0,4.118293e+09,214912000.0,180222000.0,281688000.0,209852000.0,51630000.0,149618000.0
2022-12-27,296072000.0,649901000.0,59254000.0,1.590812e+10,1.768481e+09,205669000.0,1.743574e+09,369873000.0,658389000.0,464900000.0,...,735917000.0,113314000.0,547248000.0,4.118293e+09,214912000.0,180222000.0,281688000.0,209852000.0,51630000.0,149618000.0
2022-12-28,296072000.0,649901000.0,59254000.0,1.590812e+10,1.768481e+09,205669000.0,1.743574e+09,369873000.0,658389000.0,464900000.0,...,735917000.0,113314000.0,547248000.0,4.118293e+09,214912000.0,180222000.0,281688000.0,209852000.0,51630000.0,149618000.0


In [77]:
df_shares_outstanding_clean

tic,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,347935000.0,219297000.0,73364000.0,9.388190e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100824000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
2013-01-03,347935000.0,219297000.0,73364000.0,9.388190e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100824000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
2013-01-04,347935000.0,219297000.0,73364000.0,9.388190e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100867000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
2013-01-07,347935000.0,219297000.0,73364000.0,9.388190e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100867000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
2013-01-08,347935000.0,219297000.0,73364000.0,9.390390e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100867000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,296072000.0,649901000.0,59254000.0,1.590812e+10,1.768481e+09,205669000.0,1.743574e+09,369873000.0,658389000.0,464900000.0,...,735917000.0,113314000.0,547248000.0,4.118293e+09,214912000.0,180222000.0,281688000.0,209852000.0,51630000.0,149618000.0
2022-12-27,296072000.0,649901000.0,59254000.0,1.590812e+10,1.768481e+09,205669000.0,1.743574e+09,369873000.0,658389000.0,464900000.0,...,735917000.0,113314000.0,547248000.0,4.118293e+09,214912000.0,180222000.0,281688000.0,209852000.0,51630000.0,149618000.0
2022-12-28,296072000.0,649901000.0,59254000.0,1.590812e+10,1.768481e+09,205669000.0,1.743574e+09,369873000.0,658389000.0,464900000.0,...,735917000.0,113314000.0,547248000.0,4.118293e+09,214912000.0,180222000.0,281688000.0,209852000.0,51630000.0,149618000.0


In [78]:
df_volume_clean.shape

(2518, 463)

In [79]:
df_shares_outstanding_clean.shape

(2539, 463)

In [80]:
df_daily_returns_clean.shape

(2518, 463)

In [81]:
# removing rows from df_shares_outstanding_clean and df_volume_clean that are not in df_daily_returns_clean, so dataframes are same shape
mask_1 = df_shares_outstanding_clean.index.isin(df_volume_clean.index)
df_shares_outstanding_clean = df_shares_outstanding_clean[mask_1]
mask_2 = df_shares_outstanding_clean.index.isin(df_daily_returns_clean.index)
df_shares_outstanding_clean = df_shares_outstanding_clean[mask_2]
df_volume_clean = df_volume_clean[mask_2]
assert df_shares_outstanding_clean.shape == df_volume_clean.shape == df_daily_returns_clean.shape


In [82]:
df_shares_outstanding_clean

tic,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,347935000.0,219297000.0,73364000.0,9.388190e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100824000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
2013-01-03,347935000.0,219297000.0,73364000.0,9.388190e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100824000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
2013-01-04,347935000.0,219297000.0,73364000.0,9.388190e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100867000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
2013-01-07,347935000.0,219297000.0,73364000.0,9.388190e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100867000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
2013-01-08,347935000.0,219297000.0,73364000.0,9.390390e+08,1.580670e+09,235476000.0,1.580668e+09,136551000.0,644334000.0,495055000.0,...,541532000.0,100867000.0,487620000.0,4.559343e+09,141929000.0,185794000.0,451809000.0,173502000.0,50858000.0,184182000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,296072000.0,649901000.0,59254000.0,1.590812e+10,1.768481e+09,205669000.0,1.743574e+09,369873000.0,658389000.0,464900000.0,...,735917000.0,113314000.0,547248000.0,4.118293e+09,214912000.0,180222000.0,281688000.0,209852000.0,51630000.0,149618000.0
2022-12-27,296072000.0,649901000.0,59254000.0,1.590812e+10,1.768481e+09,205669000.0,1.743574e+09,369873000.0,658389000.0,464900000.0,...,735917000.0,113314000.0,547248000.0,4.118293e+09,214912000.0,180222000.0,281688000.0,209852000.0,51630000.0,149618000.0
2022-12-28,296072000.0,649901000.0,59254000.0,1.590812e+10,1.768481e+09,205669000.0,1.743574e+09,369873000.0,658389000.0,464900000.0,...,735917000.0,113314000.0,547248000.0,4.118293e+09,214912000.0,180222000.0,281688000.0,209852000.0,51630000.0,149618000.0


In [83]:
# reference: Wharton Research Data Services. "WRDS" wrds.wharton.upenn.edu, accessed 2023-01-16.

In [84]:
# Plotting mean (adj close) daily returns, averaged per day across all stocks:
df_daily_returns_clean.mean(1).plot(title='Mean Daily Returns across all stocks')

In [85]:
# Plotting mean daily volume, averaged per day across all stocks:
df_volume_clean.mean(1).plot(title='Mean Daily Volume across all stocks')


In [86]:
# Plotting Total daily volume, summed per day across all stocks:
df_volume_clean.sum(1).plot(title='Total Daily Volume across all stocks')

In [87]:
df_shares_outstanding_clean.sum(1).plot(title='Total Shares Outstanding across all stocks')

# Getting stock volume data using WRDS plugin:

conn = wrds.Connection(wrds_username='nitainijholt')

In [88]:
# saving the dataframes to csv files
df_daily_returns_clean.to_csv('data/df_daily_returns_clean.csv')
df_volume_clean.to_csv('data/df_volume_clean.csv')
df_shares_outstanding_clean.to_csv('data/df_shares_outstanding_clean.csv')
df_volume_clean_wrds.to_csv('data/df_volume_clean_WRDS.csv')

# Code below is unused so far

# Making a DataFrame containing Daily volume

In [89]:
df['Volume'].columns

Index(['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABT', 'ACGL', 'ACN', 'ADBE',
       ...
       'WYNN', 'XEL', 'XOM', 'XRAY', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZION',
       'ZTS'],
      dtype='object', length=503)

In [90]:
col_with_missing_values = {}
for col in df['Volume'].columns:
    sum_of_missing_values = df['Adj Close'][col].isnull().sum()
    if sum_of_missing_values > 0:
        col_with_missing_values[col] = sum_of_missing_values
        print(col)

ALLE
ANET
BF.B
BRK.B
CARR
CDAY
CDW
CEG
CFG
CTLT
CTVA
CZR
DOW
ETSY
FOX
FOXA
FTV
HLT
HPE
HWM
INVH
IQV
IR
KEYS
KHC
LW
MRNA
NCLH
NWS
NWSA
OGN
OTIS
PAYC
PYPL
QRVO
SEDG
SYF
VICI
WRK
ZTS
